# Data Augmentation Using Generative Adversarial Networks (GANs)

Notebook to perform data augmentation using Generative Adversarial Networks (GANs). This notebook must be run after running the notebook `prepare_dataset_and_create_project_structure.ipynb` that should be available in the same directory.

---
## Prerequisites

1. Ensure that the Cityscapes dataset is downloaded and placed in the directory named **dataset**. More information about the dataset can be found in the notebook `prepare_dataset_and_create_project_structure.ipynb` that should be available in the same directory. 
2. Run all the cells in the notebook `prepare_dataset_and_create_project_structure.ipynb` to understand the dataset, process the dataset, create the project structure required to ensure correct results from the project.

### 1. Check Python version

It is crucial to ensure that the notebook runs on the correct version of Python to guarantee proper functionality. 

In [1]:
import platform
assert (platform.python_version_tuple()[:2] >= ('3','7')), "[ERROR] The notebooks are tested on Python 3.7 and higher. Please updated your Python to evaluate the code"

### 2. Check Notebook server has access to all required resources

In [2]:
from pathlib import Path

dataset_folder = Path("dataset")
dataset_folder = Path.joinpath(Path.cwd(), dataset_folder)

if not dataset_folder.exists():
    raise FileNotFoundError("[ERROR] Add `{}` folder in the current directory (`{}`)".format(dataset_folder.name, Path.cwd()))

In [3]:
dataset_preparation_notebook = Path("prepare_dataset_and_create_project_structure.ipynb")
dataset_preparation_notebook = Path.joinpath(Path.cwd(), dataset_preparation_notebook)

if not dataset_preparation_notebook.exists():
    raise FileNotFoundError("[ERROR] The notebook `{}` is unavailable in the current directory (`{}`). Please download and run the notebook `{}` before this notebook to ensure proper results.".format(dataset_preparation_notebook.name, Path.cwd(), dataset_preparation_notebook.name))

In [4]:
test_dataset = Path.joinpath(dataset_folder, "test_dataset")
test_dataset_A = Path.joinpath(test_dataset, "A")
test_dataset_B = Path.joinpath(test_dataset, "B")

test_dataset_overall = [test_dataset, test_dataset_A, test_dataset_B]

for dataset in test_dataset_overall:
    if not dataset.exists():
        raise FileNotFoundError("[ERROR] The folder `{}` is unavailable. Please run the notebook `prepare_dataset_and_create_project_structure.ipynb` available in the current directory (`{}`) before running this notebook.".format(dataset.name, Path.cwd()))

In [5]:
training_dataset = Path.joinpath(dataset_folder, "training_dataset")
training_dataset_A = Path.joinpath(training_dataset, "A")
training_dataset_B = Path.joinpath(training_dataset, "B")

training_dataset_overall = [training_dataset, training_dataset_A, training_dataset_B]

for dataset in training_dataset_overall:
    if not dataset.exists():
        raise FileNotFoundError("[ERROR] The folder `{}` is unavailable. Please run the notebook `prepare_dataset_and_create_project_structure.ipynb` available in the current directory (`{}`) before running this notebook.".format(dataset.name, Path.cwd()))

In [6]:
validatation_dataset = Path.joinpath(dataset_folder, "validatation_dataset")
validatation_dataset_A = Path.joinpath(validatation_dataset, "A")
validatation_dataset_B = Path.joinpath(validatation_dataset, "B")

validatation_dataset_overall = [validatation_dataset, validatation_dataset_A, validatation_dataset_B]

for dataset in validatation_dataset_overall:
    if not dataset.exists():
        raise FileNotFoundError("[ERROR] The folder `{}` is unavailable. Please run the notebook `prepare_dataset_and_create_project_structure.ipynb` available in the current directory (`{}`) before running this notebook.".format(dataset.name, Path.cwd()))

---
## Introduction

One of the biggest bottlenecks in creating generalized deep learning models is a scarcity of high-quality data. The collection of high-quality data and its conversion is expensive. Most of the data collection methods are labor-intensive and error-prone, requiring considerable editing afterward to clean the data. Since large amounts of data are needed to achieve generalized deep learning models, standard data augmentation methods are routinely used to increase the dataset's generalizability. Data augmentation methods are also used when the datasets are imbalanced, improving the model's overall performance.

Generative Adversarial Networks, popularly known as GANs, are a novel method for data augmentation. The generation of artificial training data can not only be instrumental in situations such as imbalanced data sets, but it can also be useful when the original dataset contains sensitive information. In such cases, it is then desirable to avoid using the original data as much as possible (For example, Medical data).

This report proposes a GAN architecture based on a [paper](https://arxiv.org/abs/1611.07004) from UC Berkeley to perform data augmentation using the popular image-to-image translation method. Generative Adversarial Networks trained on these methods learn the mapping from an input image to an output image and learn a loss function to train this mapping. Therefore, this approach makes it possible to apply the same generic approach to problems that traditionally require very different loss formulations. In this particular report, we demonstrate that this approach can effectively synthesize photos from label maps. To evaluate the performance of the proposed GAN architecture, we utilize a standard dataset named Cityscapes. The Cityscapes Dataset focuses on semantic understanding of urban street scenes. The dataset contains 5000 images with detailed annotations and 20000 images with coarse annotations apart from the original images. Some sample images from the dataset are presented below:

![Sample Image from Cityscape Dataset](https://www.cityscapes-dataset.com/wordpress/wp-content/uploads/2015/07/stuttgart02-2040x500.png)
![Sample Image from Cityscape Dataset](https://www.cityscapes-dataset.com/wordpress/wp-content/uploads/2015/07/stuttgart00-2040x500.png)
![Sample Image from Cityscape Dataset](https://www.cityscapes-dataset.com/wordpress/wp-content/uploads/2015/07/stuttgart04-2040x500.png)
![Sample Image from Cityscape Dataset](https://www.cityscapes-dataset.com/wordpress/wp-content/uploads/2015/07/stuttgart01-2040x500.png)

<center>Image Courtesy: Cityscapes Datatset (Link: https://www.cityscapes-dataset.com/)</center>

---
## Background Theory

With the advancements in deep learning, the most striking successes have involved discriminative models, usually those that map a high-dimensional, rich sensory input to a class label. These striking successes have primarily been based on the backpropagation and dropout algorithms, using piecewise linear units. However, deep generative models have had less impact due to the challenges of approximating many probabilistic computations that occur due to the usage of piecewise linear units in the generative context. 

Generative Adversarial Networks, popularly known as GANs, is a machine learning framework class that sidesteps these difficulties by pitting the generative model against an adversary. In other words, a GAN is a machine learning framework where two neural networks compete against each other in a zero-sum game (i.e., one network's gain is the other network's loss). The two networks in a GAN can be considered as a generator and a discriminator. The generator learns to create images that look real, while the discriminator learns to tell real images apart from fakes. Competition in this game drives both networks to improve their models until the counterfeits are indistinguishable from the genuine images. An overview of the Generative Adversarial Network is represented below:

![Overview of Generative Adversarial Network](https://developers.google.com/machine-learning/gan/images/gan_diagram.svg)

<center>Image Courtesy: Google Developers (Link: https://developers.google.com/machine-learning/gan/gan_structure)</center>

One of the most significant characteristics of GANs is the lack of loss function. GANs learn the loss function to classify if the output image is real or fake while simultaneously training a generative model to minimize this loss. This property of GANs allows it to learn a loss that adapts to the data, making them a perfect solution to a multitude of tasks that traditionally would require very different kinds of loss functions such as image-to-image translation. The image-to-image translation is the task of translating one possible representation of a scene into another, given sufficient training data. Traditionally, each image-to-image translation task has been tackled with separate, special-purpose machinery, although the setting is always the same: predict pixels from pixels. The biggest problem with this approach is the need to formulate specialized loss functions that drive the neural network to do what we want – e.g., output sharp, realistic images. Fortunately, the need for specialized loss functions is precisely eliminated with the use of GANs. 

In this report, we explore GANs in the conditional setting. Just as GANs learn a generative data model, conditional GANs (cGANs) learn a conditional generative model. In other words, just as GANs are generative models that learn a mapping from random noise vector $z$ to output image $y$, $G:\,z\,\rightarrow\,y$, the conditional GANs (cGANs) learn a mapping from observed image $x$ and random noise vector $z$, to output image $y$,$G:\,\{x,\, z\}\,\rightarrow\,y$. This ability to map from an observed image to an output image makes cGANs fitting for image-to-image translation tasks, where we condition on an input image and generate a corresponding output image. 

Finally, to understand the objective function of the conditional GANs (i.e., cGANS), let us first examine the objective function of a traditional GAN. The equation below refers to the objective function of a traditional GAN.

$$
\begin{align}
\mathbb{L}_{GAN}(G,D)\,=\,&\mathbb{E}_{y}[logD(y)]\,+\, \\
&\mathbb{E}_{x\,,\,z}[log(1\,-\,D(G(x\,,\,z)))]
\end{align}
$$

It can be observed from the Equation above that Generator $G$ tries to minimize the objective function against an adversarial $D$ that tries to maximize it. Therefore, the optimal value of G can be represented as the equation below.

$$
\begin{align}
G^{*}\,=\,arg\,min_{G}\,max_{D}\,\mathbb{L}_{GAN}(G\,,\,D)
\end{align}
$$

Based on the Equation above, the objective function of a conditional GAN can be represented as Equation below.

$$
\begin{align}
\mathbb{L}_{cGAN}(G,D)\,=\,&\mathbb{E}_{x\,,\,y}[logD(x\,,\,y)]\,+\, \\
&\mathbb{E}_{x\,,\,z}[log(1\,-\,D(x\,,\,G(x\,,\,z)))]
\end{align}
$$

Similar to the equation above, Generator $G$ tries to minimize the objective function against an adversarial $D$ that tries to maximize it. By training both $G$ and $D$ on the objective function $\mathbb{L}_{cGAN}(G\,,\,D)$ simultaneously, the optimal values for both the parameters can be calculated.

---
## Proposed Solution

To be added

### 0. Imports

In [7]:
import os
import glob
import time
import numpy as np 
import scipy as sp
from imageio import imread
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate

### 1. Load Data

In [8]:
def draw_randomized_data(sample_size, dataset_type="training"):
    """
    Function to draw randomized data samples from dataset. This function would be required while displaying training output.
    
    Parameters
    ----------
    sample_size  : Number of data samples to be loaded (type=int)
    dataset_type : Type of dataset to draw randomized data samples from (type=str) 
    
    Returns
    -------
    random_ground_truth_data  : Randomized "Ground Truth Data" drawn from the dataset "dataset_type" (type=list)
    random_input_data         : Randomized "Input Data" drawn from the dataset "dataset_type" (type=list)
    """
    
    if(dataset_type == "training"):
        ground_truth_dataset_expr = str(training_dataset_A) + "\\**\\*.jpg"
        input_dataset_expr = str(training_dataset_B) + "\\**\\*.jpg"
    else:
        ground_truth_dataset_expr = str(validatation_dataset_A) + "\\**\\*.jpg"
        input_dataset_expr = str(validatation_dataset_B) + "\\**\\*.jpg"
        
    ground_truth_dataset_paths = glob.glob(ground_truth_dataset_expr, recursive=True)
    ground_truth_dataset_paths = sorted(ground_truth_dataset_paths)
   
    input_dataset_paths = glob.glob(input_dataset_expr, recursive=True)
    input_dataset_paths = sorted(input_dataset_paths)
    
    assert (len(ground_truth_dataset_paths) == len(input_dataset_paths)), ("[ERROR] Datasets Modified After Preprocessing! Please reload the dataset from the beginning!")
        
    random_data_samples = np.random.choice(range(len(ground_truth_dataset_paths)), sample_size) # Pick any random data from the dataset
   
    random_ground_truth_data = []
    random_input_data = []
    
    for random_sample in random_data_samples:
        
        ground_truth_data_path = ground_truth_dataset_paths[random_sample]
        input_data_path = input_dataset_paths[random_sample]
        
        ground_truth_data = imread(ground_truth_data_path)
        input_data = imread(input_data_path)
        
        if((dataset_type != "training") and (np.random.random() > 0.5)):
            ground_truth_data = np.fliplr(ground_truth_data)
            input_data = np.fliplr(input_data)
        
        random_ground_truth_data.append(ground_truth_data)
        random_input_data.append(input_data)
        
    # random_ground_truth_data = np.array(random_ground_truth_data)/127.5 - 1
    # random_input_data = np.array(random_input_data)/127.5 - 1
    
    print("np.max(random_ground_truth_data) = ", np.max(random_ground_truth_data))
    print("np.min(random_ground_truth_data) = ", np.min(random_ground_truth_data))
    print("np.max(random_input_data) = ", np.max(random_input_data))
    print("np.min(random_input_data) = ", np.min(random_input_data))
    
    return random_ground_truth_data, random_input_data

draw_randomized_data(1)

np.max(random_ground_truth_data) =  253
np.min(random_ground_truth_data) =  2
np.max(random_input_data) =  255
np.min(random_input_data) =  0


([Array([[[191, 204, 160],
          [203, 215, 203],
          [207, 219, 207],
          ...,
          [ 35,  45,  34],
          [ 64,  64,  54],
          [ 44,  34,  33]],
  
         [[182, 199, 154],
          [205, 217, 205],
          [207, 220, 203],
          ...,
          [ 42,  52,  41],
          [ 64,  66,  55],
          [ 37,  29,  26]],
  
         [[176, 192, 153],
          [206, 216, 205],
          [206, 218, 204],
          ...,
          [ 49,  59,  50],
          [ 58,  64,  54],
          [ 27,  26,  22]],
  
         ...,
  
         [[ 92, 105,  87],
          [ 84,  95,  78],
          [ 82,  93,  76],
          ...,
          [ 22,  27,  23],
          [ 21,  26,  22],
          [ 47,  54,  46]],
  
         [[ 93, 104,  87],
          [ 84,  95,  78],
          [ 82,  93,  76],
          ...,
          [ 22,  27,  23],
          [ 20,  25,  21],
          [ 45,  52,  44]],
  
         [[ 93, 104,  87],
          [ 84,  95,  78],
          [ 82,  93,  76

In [9]:
def load_batches(batch_size, dataset_type="training"):
    """
    Function to load batches. This function would be required while training the neural network.
    
    Parameters
    ----------
    batch_size   : Size of the batches to be loaded (type=int)
    dataset_type : Type of dataset to load batches from (type=str) 
    
    Yeilds (Returns a Generator and Not an Iterator [IMPORTANT!])
    ------
    ground_truth_data_batch  : Batch of "Ground Truth Data" loaded from the dataset "dataset_type" (type=list)
    input_data_batch         : Batch of "Input Data" loaded from the dataset "dataset_type" (type=list)
    """
    
    if(dataset_type == "training"):
        ground_truth_dataset_expr = str(training_dataset_A) + "\\**\\*.jpg"
        input_dataset_expr = str(training_dataset_B) + "\\**\\*.jpg"
    else:
        ground_truth_dataset_expr = str(validatation_dataset_A) + "\\**\\*.jpg"
        input_dataset_expr = str(validatation_dataset_B) + "\\**\\*.jpg"
        
    ground_truth_dataset_paths = glob.glob(ground_truth_dataset_expr, recursive=True)
    ground_truth_dataset_paths = sorted(ground_truth_dataset_paths)
   
    input_dataset_paths = glob.glob(input_dataset_expr, recursive=True)
    input_dataset_paths = sorted(input_dataset_paths)
    
    assert (len(ground_truth_dataset_paths) == len(input_dataset_paths)), ("[ERROR] Datasets Modified After Preprocessing! Please reload the dataset from the beginning!")
    
    print(len(ground_truth_dataset_paths))
    print(int(len(ground_truth_dataset_paths)/batch_size))
    
    # for batch_iterator in range(int(len(ground_truth_dataset_paths)/batch_size) - 1):
    for batch_iterator in range(batch_size - 1):

        ground_truth_dataset_paths_batch = ground_truth_dataset_paths[(batch_iterator)*batch_size:(batch_iterator+1)*batch_size]
        input_dataset_paths_batch = input_dataset_paths[(batch_iterator)*batch_size:(batch_iterator+1)*batch_size]

        ground_truth_data_batch = []
        input_data_batch = []

        for (ground_truth_data_path_iterator, input_data_path_iterator) in zip(ground_truth_dataset_paths_batch, input_dataset_paths_batch):

            ground_truth_data = imread(ground_truth_data_path_iterator)
            input_data = imread(input_data_path_iterator)
           
            if((dataset_type != "training") and (np.random.random() > 0.5)):
                ground_truth_data = np.fliplr(ground_truth_data)
                input_data = np.fliplr(input_data)
        
            ground_truth_data_batch.append(ground_truth_data)
            input_data_batch.append(input_data)
            
        # random_ground_truth_data = np.array(random_ground_truth_data)/127.5 - 1
        # random_input_data = np.array(random_input_data)/127.5 - 1
        
        yield ground_truth_data_batch, input_data_batch

### 2. Define cGAN Architecture

In [10]:
def build_generator():
    """
    Closure Function to Build A Generator for cGAN.
    
    Parameters
    ----------
    None
    
    Returns
    -------
    Model   : Generator Model for cGAN (type=tf.keras.models.Model)
    """
    def conv2d(input_layer, filters, filter_shape=4, batch_normalization=True):
        """
        Nested Function to create a 2D-Convolution Layer
        
        Parameters
        ----------
        input_layer         : Input Layer to the 2D-Convolution Layer
        filters             : Number of filters in the convolution
        filter_shape        : Size of the 2D convolution window
        batch_normalization : Flag to set Batch Normalization
        
        Returns
        -------
        d    : 2D-Convolution Layer
        """
        d = Conv2D(filters, kernel_size=filter_shape, strides=2, padding='same')(input_layer)
        d = LeakyReLU(alpha=0.2)(d)
        if(batch_normalization == True):
            d = BatchNormalization(momentum=0.8)(d)
            
        return d
    
    def deconv2d(input_layer, skip_input, filters, filter_shape=4, dropout_rate=0):
        """
        Nested Function to create a 2D-Deconvolution Layer
        
        Parameters
        ----------
        input_layer   : Input Layer to the 2D-Convolution Layer
        skip_input    : Input Layer to be Skipped (i.e., Inputs of this layer are concatenated)
        filters       : Number of filters in the convolution
        filter_shape  : Size of the 2D convolution window
        dropout_rate  : Dropout Rate
        
        Returns
        -------
        u    : 2D-Deconvolution Layer
        """
        u = UpSampling2D(size=2)(input_layer)
        u = Conv2D(filters, kernel_size=filter_shape, strides=1, padding='same', activation='relu')(u)
        if dropout_rate:
            u = Dropout(dropout_rate)(u)
        u = BatchNormalization(momentum=0.8)(u)
        u = Concatenate()([u, skip_input]) #skip connection
        
        return u
    
    
    d0 = Input(shape=image_shape)

    # Downsampling
    d1 = conv2d(d0, gf, batch_normalization=False)
    d2 = conv2d(d1, gf*2)
    d3 = conv2d(d2, gf*4)
    d4 = conv2d(d3, gf*8)
    d5 = conv2d(d4, gf*8)
    d6 = conv2d(d5, gf*8)
    d7 = conv2d(d6, gf*8)

    # Upsampling
    u1 = deconv2d(d7, d6, gf*8)
    u2 = deconv2d(u1, d5, gf*8)
    u3 = deconv2d(u2, d4, gf*8)
    u4 = deconv2d(u3, d3, gf*4)
    u5 = deconv2d(u4, d2, gf*2)
    u6 = deconv2d(u5, d1, gf)

    u7 = UpSampling2D(size=2)(u6)
    generated_image = Conv2D(channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u7)

    return Model(d0, generated_image)

In [11]:
def build_discriminator():
    """
    Closure Function to Build A Discriminator for cGAN.
    
    Parameters
    ----------
    None
    
    Returns
    -------
    Model   : Discriminator Model for cGAN (type=tf.keras.models.Model)
    """
    def d_layer(input_layer, filters, filter_shape=4, batch_normalization=True):
        """
        Nested Function to create a single layer discriminator
        
        Parameters
        ----------
        input_layer         : Input Layer to the 2D-Convolution Layer
        filters             : Number of filters in the convolution
        filter_shape        : Size of the 2D convolution window
        batch_normalization : Flag to set Batch Normalization
        
        Returns
        -------
        d    : Discriminator layer
        """
        d = Conv2D(filters, kernel_size=filter_shape, strides=2, padding='same')(input_layer)
        d = LeakyReLU(alpha=0.2)(d)
        if batch_normalization:
            d = BatchNormalization(momentum=0.8)(d)
            
        return d
    
    ground_truth_image = Input(shape=image_shape)
    input_image = Input(shape=image_shape)

    # Concatenate image and conditioning image by channels to produce input
    combined_images = Concatenate(axis=-1)([ground_truth_image, input_image])

    d1 = d_layer(combined_images, df, batch_normalization=False)
    d2 = d_layer(d1, df*2)
    d3 = d_layer(d2, df*4)
    d4 = d_layer(d3, df*8)

    validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

    return Model([ground_truth_image, input_image], validity)

### 3. Training

In [12]:
# Input shape
image_rows = 128
image_columns = 128
channels = 3
image_shape = (image_rows, image_columns, channels)

# Calculate output shape of D (PatchGAN)
patch = int(image_rows / 2**4)
discriminator_patch = (patch, patch, 1)

# Number of filters in the first layer of G and D
gf = 64
df = 64

optimizer = Adam(0.0002, 0.5)

# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

# Build the generator
generator = build_generator()

# Input images and their conditioning images
ground_truth_image = Input(shape=image_shape)
input_image = Input(shape=image_shape)

# By conditioning on B generate a fake version of A
generated_ground_truth_image = generator(input_image)

# For the combined model we will only train the generator
discriminator.trainable = False

# Discriminators determines validity of translated images / condition pairs
valid = discriminator([generated_ground_truth_image, input_image])

gan = Model(inputs=[ground_truth_image, input_image], outputs=[valid, generated_ground_truth_image])
gan.compile(loss=['mse', 'mae'], loss_weights=[1, 100], optimizer=optimizer)

In [13]:
# Function to display output

def display_output(output_data_type, number_of_outputs=3):
    
    real_ground_truth_images, input_images = draw_randomized_data(number_of_outputs, dataset_type=output_data_type)
    
    # print(real_ground_truth_images)
    print(type(real_ground_truth_images))
    print(len(real_ground_truth_images))
    print(real_ground_truth_images[0].shape)
    
    # print(input_images)
    print(type(input_images))
    print(len(input_images))
    print(input_images[0].shape)
    
    generated_ground_truth_image = generator.predict(input_images)
    
    print(generated_ground_truth_image)
    print(type(generated_ground_truth_image))
    print(generated_ground_truth_image.shape)
    
    overall_generated_images = np.concatenate(input_images, generated_ground_truth_image, real_ground_truth_images)
    
    print(overall_generated_images)
    print(type(overall_generated_images))
    print(overall_generated_images.shape)
    
    # Rescale Image
    overall_generated_images = 0.5 * overall_generated_images + 0.5
    
    titles = ["Input", "Output", "Ground Truth"]
    
    f, axarr = plt.subplots(3, number_of_outputs, figsize=(20,30))
    
    for row_iterator in range(3):
        for column_iterator in range(number_of_outputs):
            axarr[row_iterator, column_iterator].imshow(overall_generated_images[row_iterator + column_iterator])
            axarr[row_iterator, column_iterator].set_title(titles[row_iterator])
            axarr[row_iterator, column_iterator].axis("off")
    
    plt.show()
    plt.close()

In [14]:
# Function to perform training
def train_neural_network(epochs, batch_size, display_interval=10):
    
    start_time = time.time()
    print("cGAN Training started at {}".format(time.asctime(time.localtime(start_time))))
    
    # Calculate Batch Shape and Target Values
    batch_shape = (batch_size,) + discriminator_patch    # Joining Tuples to create a batch shape of dimensions: (batch, height, width, channels)
    target_real_images = np.ones(batch_shape)
    target_fake_images = np.zeros(batch_shape)
    
    # Initialize Generaor and Discriminator Losses
    generator_loss_overall = []
    discriminator_loss_overall = []
    
    for current_epoch in range(epochs):
        
        for batch_iterator, (ground_truth_images, input_images) in enumerate(load_batches(batch_size)):
        
            print("len(ground_truth_images)= ", len(ground_truth_images))
            print("ground_truth_images[0].shape= ", ground_truth_images[0].shape)
            print("len(input_images)= ", len(input_images))
            print("input_images[0].shape= ", input_images[0].shape)
            
            # Generate Fake Ground Truth Images
            generated_ground_truth_image = generator.predict(input_images)
            
            # Train Discriminator
            discriminator_loss_real_ground_truth = discriminator.train_on_batch([ground_truth_images, input_images], target_real_images)
            
            print(discriminator_loss_real_ground_truth)
            print(type(discriminator_loss_real_ground_truth))
            print(discriminator_loss_real_ground_truth.shape)
            
            
            discriminator_loss_fake_ground_truth = discriminator.train_on_batch([generated_ground_truth_image, input_images], target_fake_images)
            
            print(discriminator_loss_fake_ground_truth)
            print(type(discriminator_loss_fake_ground_truth))
            print(discriminator_loss_fake_ground_truth.shape)
            
            discriminator_loss_current = 0.5*np.add(discriminator_loss_real_ground_truth, discriminator_loss_fake_ground_truth)
            
            print(discriminator_loss_current)
            print(type(discriminator_loss_current))
            print(discriminator_loss_current.shape)
            
            # Train Generator
            generator_loss_current = gan.train_on_batch([ground_truth_images, input_images], [target_real_images, ground_truth_images])
            
            print(generator_loss_current)
            print(type(generator_loss_current))
            print(generator_loss_current.shape)
        
            # Calculate Elapsed Time
            current_time = time.time()    
            elapsed_time = current_time - start_time  
            
        # Append Losses
        generator_loss_overall.append(generator_loss_current)
        discriminator_loss_overall.append(discriminator_loss_current)
        
        # Display Results
        print(f"Epoch: {current_epoch + 1}/{epochs}, Generator Loss: {generator_loss_current:.2f}, Discriminator Loss: {discriminator_loss_current:.2f}, Elapsed Time: {elapsed_time:.2f}")
        
        # Display Images
        if(current_epoch % display_interval == 0):
            display_output(output_data_type="training")

In [15]:
train_neural_network(epochs=1, batch_size=32, display_interval=1)

cGAN Training started at Mon Oct 26 07:24:51 2020
2975
92
len(ground_truth_images)=  32
ground_truth_images[0].shape=  (128, 128, 3)
len(input_images)=  32
input_images[0].shape=  (128, 128, 3)


ValueError: in user code:

    C:\Users\soura\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:1462 predict_function  *
        return step_function(self, iterator)
    C:\Users\soura\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\soura\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\soura\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\soura\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\s

---
## Results

---
## Conclusions